## **N^2 - 1 Puzzle Problem**
Write a Python/ Java/ C/ C++ program to solve the (n^2 − 1)-puzzle problem using the following search algorithms:
- Breadth-First Search (BFS)
- Depth-First Search (DFS)
- Iterative Deepening (ID)

### 1. Breadth-First Search (BFS)

In [ ]:
from collections import deque
import copy
import time

class Node:
    def __init__(self, state, parent=None, move=None, depth=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.size = len(state)
        
    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.state])
        
    def get_blank_pos(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.state[i][j] == 0:
                    return i, j
                    
    def get_path(self):
        path = []
        current = self
        while current.parent:
            path.append(current.move)
            current = current.parent
        return path[::-1]

def get_inversions(state):
    flat = [x for row in state for x in row if x != 0]
    inv_count = 0
    for i in range(len(flat)):
        for j in range(i + 1, len(flat)):
            if flat[i] > flat[j]:
                inv_count += 1
    return inv_count

def is_solvable(state):
    n = len(state)
    inv_count = get_inversions(state)
    
    # Find blank position from bottom
    blank_row = 0
    for i in range(n-1, -1, -1):
        for j in range(n):
            if state[i][j] == 0:
                blank_row = n - i
                break
    
    if n % 2 == 1:
        return inv_count % 2 == 0
    else:
        if blank_row % 2 == 0:
            return inv_count % 2 == 1
        else:
            return inv_count % 2 == 0

def get_neighbors(node):
    moves = [(0, 1, 'R'), (1, 0, 'D'), (0, -1, 'L'), (-1, 0, 'U')]
    neighbors = []
    blank_i, blank_j = node.get_blank_pos()
    
    for di, dj, move in moves:
        new_i, new_j = blank_i + di, blank_j + dj
        if 0 <= new_i < node.size and 0 <= new_j < node.size:
            new_state = copy.deepcopy(node.state)
            new_state[blank_i][blank_j], new_state[new_i][new_j] = \
                new_state[new_i][new_j], new_state[blank_i][blank_j]
            neighbors.append(Node(new_state, node, move, node.depth + 1))
            
    return neighbors

def state_to_string(state):
    return ''.join(map(str, [x for row in state for x in row]))

def solve_bfs(initial_state):
    if not is_solvable(initial_state):
        return None
        
    initial_node = Node(initial_state)
    if state_to_string(initial_state) == '123456780':  # Goal state for 3x3
        return initial_node.get_path()
        
    queue = deque([initial_node])
    visited = {state_to_string(initial_state)}
    
    while queue:
        current = queue.popleft()
        
        for neighbor in get_neighbors(current):
            state_str = state_to_string(neighbor.state)
            
            if state_str == '123456780':  # Goal state
                return neighbor.get_path()
                
            if state_str not in visited:
                visited.add(state_str)
                queue.append(neighbor)
                
    return None

def main():
    # Example usage
    initial_state = [
        [1, 2, 3],
        [4, 0, 6],
        [7, 5, 8]
    ]
    
    print("Initial state:")
    print(Node(initial_state))
    
    if not is_solvable(initial_state):
        print("Puzzle is not solvable!")
        return
        
    start_time = time.time()
    solution = solve_bfs(initial_state)
    end_time = time.time()
    
    if solution:
        print(f"\nSolution found in {len(solution)} moves:")
        print(' '.join(solution))
        print(f"Time taken: {end_time - start_time:.3f} seconds")
    else:
        print("No solution found!")

if __name__ == "__main__":
    main()